In [1]:
# Import necessary libraries
import sys
import os

sys.path.append(os.path.abspath('../scripts'))


In [2]:
# Import the necessary modules
import os
from dotenv import load_dotenv
from telethon import TelegramClient, types  # Import types for media handling
import json

# Step 1: Load environment variables from the .env file
load_dotenv()

# Step 2: Fetch API ID and API Hash from environment variables
api_id = os.getenv('TELEGRAM_API_ID')
api_hash = os.getenv('TELEGRAM_API_HASH')

# Step 3: Initialize the Telegram client (with extended timeout)
client = TelegramClient('scraper_session_alt', api_id, api_hash, timeout=60)

# Step 4: Define a function to fetch messages from a channel
async def fetch_messages(channel_link, limit=100):
    messages_data = []

    async for message in client.iter_messages(channel_link, limit=limit):
        message_dict = {
            "sender": message.sender_id,
            "timestamp": message.date.isoformat(),
            "content": message.message,
            "media": None  # Initialize with None, updated later if media exists
        }
        
        # Check if the message contains media
        if message.media:
            # Handle different media types
            if isinstance(message.media, types.MessageMediaPhoto):
                media_file = await client.download_media(message.media, file=f"./media/photo_{message.id}")
                message_dict["media"] = media_file  # Save the path to the downloaded photo
            elif isinstance(message.media, types.MessageMediaDocument):
                media_file = await client.download_media(message.media, file=f"./media/doc_{message.id}")
                message_dict["media"] = media_file  # Save the path to the downloaded document
            elif isinstance(message.media, types.MessageMediaWebPage):
                # Handle web pages or other media types differently if needed
                message_dict["media"] = f"Web page URL: {message.media.webpage.url}" if message.media.webpage else "Unknown webpage"

        messages_data.append(message_dict)

    return messages_data

# Step 5: Define a function to fetch data from multiple channels
async def fetch_all_channels(channels, limit=100):
    for channel in channels:
        print(f"Fetching data from {channel}...")
        data = await fetch_messages(channel, limit)
        file_name = f'{channel.split("/")[-1]}_messages.json'
        with open(file_name, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Saved {len(data)} messages from {channel} to {file_name}")

# Step 6: List the Telegram channels you want to scrape
channels = [
    'https://t.me/chuchushoes'
]

# Step 7: Use await directly in Jupyter notebook
async def main():
    async with client:
        await fetch_all_channels(channels, limit=500)

# Instead of client.loop.run_until_complete(main()), just use await main()
await main()


Fetching data from https://t.me/chuchushoes...


NameError: name 'types' is not defined

In [10]:
import os

if not os.path.exists('processed_messages.json'):
    with open('processed_messages.json', 'w') as f:
        f.write('')  # Create an empty file if i

In [ ]:
# Import necessary libraries
import nest_asyncio
import asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Import your TelegramScraper class and run_scraper function
from tele_fetch import TelegramScraper, run_scraper  # Ensure tele_fetch.py is in the same directory or provide the correct path

# Define a function to run the scraper with timeout
async def main():
    await run_scraper(channel='https://t.me/chuchushoes', timeout=600)  # Run the scraper for 10 minutes

# Run the main function in the Jupyter Notebook
await main()




In [ ]:
import pandas as pd

# Load the processed messages from the JSON file
data = pd.read_json('processed_messages.json', lines=True)

# Display the first few rows of the data
data.head()
